### Финальное тестирование модели на тестовой выборке


In [1]:
!pip install catboost

In [2]:
import numpy as np
import pandas as pd
import pickle

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

In [3]:
data = pd.read_csv("cian_rent_Moscow_final.csv")
data.head()

,floor,floors_count,rooms_count,total_meters,price_per_month,district,street,house_number,underground,lat,lon,geo_quality,dist_to_center_km,dist_to_metro_km,parks_1km,groceries_top_1km,schools_1km,pharmacies_top_1km,clinics_1km
0,3,5,1,30.0,30000,Северное Измайлово,Сиреневый бульвар,7,Щелковская,55.802980,37.775445,1,11.387233,1.623468,0,11,4,0,4
1,2,34,0,55.7,25000,Войковский,Адмирала Макарова,6Ак1,Водный стадион,55.833869,37.491056,0,12.054178,0.609563,0,6,1,0,0
2,5,9,1,50.0,30000,Нагорный,Варшавское шоссе,71К1,Варшавская,55.660412,37.620267,1,10.188199,0.788797,0,2,5,0,4
3,1,5,1,32.0,30000,Лосиноостровский,Анадырский проезд,41,Бабушкинская,55.870534,37.693488,1,14.007041,1.818472,0,3,3,1,0
4,10,12,1,23.0,29998,НАО (Новомосковский),Яворки,1к1,Филатов луг,55.509092,37.345487,1,31.956738,10.803737,0,0,1,0,0


In [4]:
X = data.drop("price_per_month", axis = 1)
y = data["price_per_month"]

In [5]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, random_state = 42, train_size = 0.8, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state = 42, test_size = 0.5, shuffle = True)

In [6]:
with open('final_rent_model_catboost.pkl', 'rb') as file:
    model = pickle.load(file)

In [7]:
y_pred = np.expm1(model.predict(X_test))

MAE = round(mean_absolute_error(y_test, y_pred), 2)
R2 = round(r2_score(y_test, y_pred), 4)
MAPE = round(mean_absolute_percentage_error(y_test, y_pred), 4)


print(f"MAE: {MAE}")
print(f"R2-Score: {R2}")
print(f"MAPE: {MAPE}")

MAE: 21312.2
R2-Score: 0.8911
MAPE: 0.1631


## Финальный результат на тестовой выборке

**Метрики качества:**

| Метрика | Значение  | Интерпретация |
|---------|-----------|---------------|
| MAE     | 21 312.2 ₽ | Средняя абсолютная ошибка в денежных единицах. |
| R²      | 0.8911     | Доля объяснённой дисперсии целевой переменной. |
| MAPE    | 16.31%    | Средняя относительная ошибка прогноза. |

---

### Интерпретация:
- **Стабильность**: Разница с валидацией минимальна, признаки переобучения отсутствуют.  
- **Практическая ценность**: Ошибка в ~21 тыс. ₽ допустима для задач ценообразования на рынке аренды.  
- **Сбалансированность**: Метрики согласованно отражают высокое качество модели.  

---

### Вывод:
Оптимизированная модель CatBoost, обученная с подбором гиперпараметров через Optuna, показала устойчивое качество на тестовой выборке. Результаты подтверждают, что модель готова к практическому применению и способна прогнозировать стоимость аренды недвижимости.